# Research Project Experiment

In [ ]:
# CS 420/CS 527 Lab 2: Genetic Algorithms in LEAP 
# Author: Catherine Schuman
# February 2022

import os
import numpy as np
from toolz import pipe

from leap_ec import Individual, context, test_env_var
from leap_ec import ops, probe, util
from leap_ec.decoder import IdentityDecoder
from leap_ec.binary_rep.problems import MaxOnes
from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import mutate_bitflip
from leap_ec.binary_rep.problems import ScalarProblem
import argparse
import sys

# Implementation of a custom problem
class Lab2Problem(ScalarProblem):
    def __init__(self):
        super().__init__(maximize=True)
        
    def evaluate(self, ind):
        length_genome = len(ind)
        x=0
        for exp,b in enumerate(ind):
            x+= (int(b) * (2 ** (length_genome-exp-1)))
        fitness = (x / ((2 ** length_genome) - 1)) ** 10
        return fitness


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description="Lab 2: Genetic Algorithms")
#     parser.add_argument("--n", default=50, help="population size", type=int)
#     parser.add_argument("--p_m", default=0.01, help="probability of mutation", type=float)
#     parser.add_argument("--p_c", default=0.3, help="probability of crossover", type=float)
#     parser.add_argument("--trn_size", default=2, help="tournament size", type=int)
#     parser.add_argument("--csv_output", required=True, help="csv output file name", type=str)
#     args = parser.parse_args()    

N = 25
p_m = 0.01
p_c = 0.3
trn_size = 3

max_generation = 30
l = 40
parents = Individual.create_population(N,
                                       initialize=create_binary_sequence(
                                           l),
                                       decoder=IdentityDecoder(),
                                       problem=Lab2Problem())

# Evaluate initial population
parents = Individual.evaluate_population(parents)

generation_counter = util.inc_generation()
out_f = open(args.csv_output, "w")
while generation_counter.generation() < max_generation:
    offspring = pipe(parents,
                     ops.tournament_selection(k=trn_size),
                     ops.clone,
                     mutate_bitflip(probability=p_m),
                     ops.UniformCrossover(p_xover=p_c),
                     ops.evaluate,
                     ops.pool(size=len(parents)),  # accumulate offspring
                     probe.AttributesCSVProbe(stream=out_f, do_fitness=True, do_genome=True)
                    )
    
    parents = offspring
    generation_counter()  # increment to the next generation

out_f.close()


In [ ]:
# SNN Practice with MNIST and Rate Encoding
import torch
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load one MNIST image (as grayscale 28x28)
transform = transforms.Compose([transforms.ToTensor()])
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
image, label = mnist[0]  # Just take the first image for example
image = image.squeeze()  # Remove extra channel dimension (1, 28, 28) -> (28, 28)

# Step 2: Show the original MNIST image
plt.imshow(image, cmap='gray')
plt.title(f"Original MNIST Image - Label: {label}")
plt.axis('off')
plt.show()

# Step 3: Normalize pixel values to [0, 1] to use as firing probabilities
normalized_image = image.numpy()  # Convert tensor to numpy array
normalized_image = normalized_image / normalized_image.max()  # Ensure values between 0 and 1

# Step 4: Encode to spikes over T time steps
T = 100  # Total simulation time steps
spike_train = np.zeros((T, 28, 28))

for t in range(T):
    # Flip a coin for each pixel: will it spike at time t?
    spikes = np.random.rand(28, 28) < normalized_image
    spike_train[t] = spikes

# Step 5: Visualize spike activity over time for one pixel
pixel_row, pixel_col = 14, 14  # Pick center pixel
pixel_spike_times = spike_train[:, pixel_row, pixel_col]

print(f"Spikes over time for pixel ({pixel_row},{pixel_col}):")
print(pixel_spike_times.astype(int))

# Step 6: Plot total number of spikes at each time step (activity curve)
total_spikes = spike_train.sum(axis=(1, 2))
plt.figure()
plt.plot(range(T), total_spikes)
plt.xlabel("Time step")
plt.ylabel("Total spikes")
plt.title("Spiking Activity Over Time")
plt.show()

# Step 7: Show a spike raster plot for a small 5x5 patch of pixels
plt.figure(figsize=(10, 5))
for i in range(5):
    for j in range(5):
        spikes = spike_train[:, 10+i, 10+j]
        spike_times = np.where(spikes == 1)[0]
        plt.scatter(spike_times, np.ones_like(spike_times) * (i*5 + j), s=10)

plt.xlabel("Time step")
plt.ylabel("Pixel Index (5x5 Patch)")
plt.title("Spike Raster Plot for 5x5 Pixels")
plt.show()